# PaniniFS Research - Benchmark Performance\n\nComparaison performance GPU Colab vs système local\n\n## ⚡ Configuration Performance

In [ ]:
import time\nimport torch\nimport numpy as np\nfrom transformers import pipeline, AutoTokenizer, AutoModel\nimport psutil\nimport json\n\nclass PerformanceBenchmark:\n    def __init__(self):\n        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\n        self.results = {}\n        \n    def benchmark_text_processing(self, texts, iterations=3):\n        \"\"\"Benchmark traitement de texte\"\"\"\n        \n        # Pipeline de traitement\n        classifier = pipeline('sentiment-analysis', device=0 if self.device.type=='cuda' else -1)\n        ner = pipeline('ner', device=0 if self.device.type=='cuda' else -1)\n        \n        times = []\n        \n        for i in range(iterations):\n            start_time = time.time()\n            \n            for text in texts:\n                sentiment = classifier(text)\n                entities = ner(text)\n                \n            elapsed = time.time() - start_time\n            times.append(elapsed)\n            \n        avg_time = np.mean(times)\n        throughput = len(texts) / avg_time\n        \n        return {\n            'avg_time': avg_time,\n            'throughput': throughput,\n            'texts_processed': len(texts),\n            'device': str(self.device)\n        }\n        \n    def benchmark_large_model(self, model_name='bert-base-multilingual-cased'):\n        \"\"\"Benchmark modèle large\"\"\"\n        \n        tokenizer = AutoTokenizer.from_pretrained(model_name)\n        model = AutoModel.from_pretrained(model_name).to(self.device)\n        \n        # Test avec différentes tailles de batch\n        test_text = 'This is a test sentence for benchmarking performance.'\n        batch_sizes = [1, 8, 16, 32]\n        \n        batch_results = {}\n        \n        for batch_size in batch_sizes:\n            texts = [test_text] * batch_size\n            \n            start_time = time.time()\n            \n            inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True).to(self.device)\n            \n            with torch.no_grad():\n                outputs = model(**inputs)\n                \n            elapsed = time.time() - start_time\n            \n            batch_results[batch_size] = {\n                'time': elapsed,\n                'throughput': batch_size / elapsed\n            }\n            \n        return batch_results\n        \n    def system_info(self):\n        \"\"\"Informations système\"\"\"\n        \n        info = {\n            'device': str(self.device),\n            'python_version': sys.version,\n            'torch_version': torch.__version__,\n            'cpu_count': psutil.cpu_count(),\n            'memory_gb': round(psutil.virtual_memory().total / (1024**3), 2)\n        }\n        \n        if torch.cuda.is_available():\n            info.update({\n                'gpu_name': torch.cuda.get_device_name(),\n                'gpu_memory_gb': round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 2),\n                'cuda_version': torch.version.cuda\n            })\n            \n        return info\n\n# Exécution benchmark\nbenchmark = PerformanceBenchmark()\nsystem_info = benchmark.system_info()\n\nprint('🖥️ INFORMATIONS SYSTÈME:')\nfor key, value in system_info.items():\n    print(f'  {key}: {value}')